In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('/data/ephemeral/home/level2-objectdetection-cv-10-1/baseline/mmdetection/configs/retinanet/retinanet_r18_fpn_1x_coco.py')

root='../../dataset/'

# epoch = 'latest'
epoch = 'epoch_3'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.test_pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
# cfg.work_dir = './work_dirs/retina'
cfg.work_dir = '/data/ephemeral/home/level2-objectdetection-cv-10-1/develop/JJH/work_dirs/retina'

cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [33]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [34]:
cfg.model

{'type': 'RetinaNet',
 'backbone': {'type': 'ResNet',
  'depth': 18,
  'num_stages': 4,
  'out_indices': (0, 1, 2, 3),
  'frozen_stages': 1,
  'norm_cfg': {'type': 'BN', 'requires_grad': True},
  'norm_eval': True,
  'style': 'pytorch',
  'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet18'}},
 'neck': {'type': 'FPN',
  'in_channels': [64, 128, 256, 512],
  'out_channels': 256,
  'start_level': 1,
  'add_extra_convs': 'on_input',
  'num_outs': 5},
 'bbox_head': {'type': 'RetinaHead',
  'num_classes': 10,
  'in_channels': 256,
  'stacked_convs': 4,
  'feat_channels': 256,
  'anchor_generator': {'type': 'AnchorGenerator',
   'octave_base_scale': 4,
   'scales_per_octave': 3,
   'ratios': [0.5, 1.0, 2.0],
   'strides': [8, 16, 32, 64, 128]},
  'bbox_coder': {'type': 'DeltaXYWHBBoxCoder',
   'target_means': [0.0, 0.0, 0.0, 0.0],
   'target_stds': [1.0, 1.0, 1.0, 1.0]},
  'loss_cls': {'type': 'FocalLoss',
   'use_sigmoid': True,
   'gamma': 2.0,
   'alpha': 0.25,
   'lo

In [35]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector

In [36]:
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /data/ephemeral/home/level2-objectdetection-cv-10-1/develop/JJH/work_dirs/retina/epoch_3.pth


In [37]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 14.8 task/s, elapsed: 328s, ETA:     0s

In [39]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.44325337 604.59955 451.63742 939.85175 101...,test/0000.jpg
1,0 0.2806192 353.98532 239.77055 743.1802 704.1...,test/0001.jpg
2,0 0.29428938 715.26636 233.76627 913.04834 332...,test/0002.jpg
3,0 0.23858114 81.75551 170.43297 945.3908 962.6...,test/0003.jpg
4,0 0.25294328 351.66983 374.91187 853.53845 749...,test/0004.jpg
